In [42]:
import pandas as pd

In [43]:
df = pd.read_csv("IMDB-Movie-Data.xlsx.csv")

In [44]:
df

,ID,Title,Genre,Director,Year,Runtime_minutes,Rating,Votes,Revenue_millions
0,1,Guardians of the Galaxy,Action,James Gunn,2014,121,8.1,757074,333.13
1,2,Prometheus,Adventure,Ridley Scott,2012,124,7.0,485820,126.46
2,3,Split,Horror,M. Night Shyamalan,2016,117,7.3,157606,138.12
3,4,Sing,Animation,Christophe Lourdelet,2016,108,7.2,60545,270.32
4,5,Suicide Squad,Action,David Ayer,2016,123,6.2,393727,325.02
...,...,...,...,...,...,...,...,...,...
995,996,Secret in Their Eyes,Crime,Billy Ray,2015,111,6.2,27585,NaN
996,997,Hostel: Part II,Horror,Eli Roth,2007,94,5.5,73152,17.54
997,998,Step Up 2: The Streets,Drama,Jon M. Chu,2008,98,6.2,70699,58.01
998,999,Search Party,Adventure,Scot Armstrong,2014,93,5.6,4881,NaN


** Question 1 

What is the 75th percentile of rating in the IMDB dataset?

In [45]:
df["Rating"].quantile(q=0.75)

7.4

** Question 2

How many NA values are there in the field ‘Revenue’?

In [46]:
df.isna().sum()

ID                    0
Title                 0
Genre                 0
Director              0
Year                  0
Runtime_minutes       0
Rating                0
Votes                 0
Revenue_millions    128
dtype: int64

*** Question 3

How many movies have revenue higher than 75 million?

In [18]:
df[df['Revenue_millions'] > 75].shape[0]

318

** Question 4

How many movies have revenue greater than 50 million but rating less than 7?

In [19]:
df[(df["Revenue_millions"] > 50) & (df["Rating"] < 7)].shape[0]

211

** Question 5

What is the total revenue generated by movies in the year 2015?

In [20]:
df[df["Year"]==2015]["Revenue_millions"].sum()

8854.119999999999

** Question 6

What is the average rating for the genre adventure in the year 2015?

In [21]:
df[(df["Year"]==2015) & (df["Genre"]=="Adventure")]["Rating"].mean()

6.8

** Question 7

What is the average duration of movies in rows 75 to 150? Please note that the rows in python start from 0.

In [101]:
df.iloc[75:150]["Runtime_minutes"].mean()

127.61333333333333

** Question 8

Which year generated the highest revenue?

In [102]:
df.groupby("Year")["Revenue_millions"].sum() 

Year
2006     3624.46
2007     4306.23
2008     5053.22
2009     5292.26
2010     5989.65
2011     5431.96
2012     6910.29
2013     7666.72
2014     7997.40
2015     8854.12
2016    11211.65
Name: Revenue_millions, dtype: float64

** Question 9

What is the maximum revenue out of (10,20,30,40,50) rows?

In [103]:
df.iloc[[10, 20, 30, 40, 50]]['Revenue_millions'].max()

936.63

** Question 10

How many movies with the genres ‘Adventure’, ‘Action’, ‘Horror’, and ‘Crime’ exist in the IMDB dataset?

In [104]:
df["Genre"].isin(["Adventure", "Action", "Horror","Crime"]).sum()

485

** Question 11

Create a genre-level report with metrics average rating, the average number of votes, and average revenue. What is the average rating of the ‘Horror’ genre? (Round to 2 decimal places)

In [105]:
df.groupby("Genre").agg({'Rating': 'mean','Votes': 'mean','Revenue_millions': 'mean'})["Rating"].round(2)

Genre
Action       6.59
Adventure    6.91
Animation    7.32
Biography    7.32
Comedy       6.49
Crime        6.81
Drama        6.95
Fantasy      5.85
Horror       5.87
Mystery      6.88
Romance      6.60
Sci-Fi       4.97
Thriller     5.96
Name: Rating, dtype: float64

*** Create a report to showcase the revenue of each movie, as % revenue concerning the total revenue of the respective genre and year of the movie.

For example if a movie ‘ABC’ has genre ‘Action’ and released in 2015, then % revenue will be 

- (Revenue of the movie ‘ABC’ *100)/ (Total revenue of the genre ‘Action’ in 2015)

 What is the  % revenue of the movie ‘Split’ in its respective genre and year?


Hint:

Create a dataset which has revenue for each genre within a year
Rename the column as ‘Total_revenue’
Merge the dataset with cleaned dataset so that total revenue is mapped to each movie
Just apply percent formula using apply(lamda)
  Revenue of movie * 100 / (Total revenue of genre and year)

In [106]:
genre_year_revenue = df.groupby(['Genre', 'Year'])['Revenue_millions'].sum().reset_index()
genre_year_revenue.rename(columns={'Revenue_millions': 'Total_revenue'}, inplace=True)

In [107]:
merged_data = pd.merge(df, genre_year_revenue, on=['Genre', 'Year'])

In [108]:
merged_data['Percentage_revenue'] = (merged_data['Revenue_millions'] * 100) / merged_data['Total_revenue']

** Question 12 

Query the merged dataset for ‘Split’ movie

In [109]:
merged_data.loc[merged_data['Title'] == 'Split', 'Percentage_revenue'].values[0]

29.42041024985622

*** Add a column ‘Votes_norm’ in the IMDB dataset using apply() function where Votes_norm is 

[Votes - min(Votes)]*10/[max(votes) - min(votes)

The above formula is the normalization formula and converts votes into a scale of 0-10.

In [115]:
min_votes = df['Votes'].min()
max_votes = df['Votes'].max()

In [116]:
def normalize_votes(row):
    return ((row['Votes'] - min_votes) * 10) / (max_votes - min_votes)

In [118]:
df['Votes_norm'] = df.apply(normalize_votes, axis=1)

** Question 13

What is the average ‘Votes_norm’ ? (Round to two decimal places)

In [119]:
df.apply(normalize_votes, axis=1).mean().round(2)

0.95

*** Add a column ‘Total_rating’  in the IMDB dataset using apply() function where Total_rating is ‘Rating’+ ‘Votes_norm’.

In [128]:
def calculate_total_rating(row):return row['Rating'] + row['Votes_norm']

** Question 14

What is the highest ‘Total_rating’ ?

In [129]:
df['Total_rating'] = df.apply(calculate_total_rating, axis=1)
highest_total_rating = df['Total_rating'].max()

In [131]:
highest_total_rating

19.0

** Question 15

Create a new column ‘Revenue_bins’ so that ‘Revenue_millions’ are categorized in buckets 0-50, 51-100, 101-150, and 150+. Which bucket has the highest number of movies?

In [68]:
bin_edges = [0, 50, 100, 150, float('inf')]

In [69]:
bin_labels = ['0-50', '51-100', '101-150', '150+']

In [70]:
pd.cut(df['Revenue_millions'], bins=bin_edges, labels=bin_labels, right=False).value_counts().idxmax()

'0-50'

** Question 16

How many directors have created movies in the highest number of genres?

In [81]:
ds = df.groupby(['Director', 'Genre'])['Title'].count().reset_index(),df.groupby('Director')['Genre'].count().reset_index()['Genre'].max()
ds

(                Director      Genre  Title
 0             Aamir Khan      Drama      1
 1    Abdellatif Kechiche      Drama      1
 2              Adam Leon     Comedy      1
 3             Adam McKay     Action      2
 4             Adam McKay  Biography      1
 ..                   ...        ...    ...
 818          Yimou Zhang     Action      1
 819     Yorgos Lanthimos     Comedy      1
 820     Yorgos Lanthimos      Drama      1
 821          Zack Snyder     Action      5
 822        Zackary Adler      Crime      1
 
 [823 rows x 3 columns],
 8)

In [84]:
len(ds)

2